In [2]:
import sys
print(sys.version)
print(sys.version_info)
print(sys.version_info < (3, 0))


num_proc = 8
batch_size = 10000

3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:13) [Clang 14.0.6 ]
sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)
False


In [3]:
import tiktoken
from datasets import load_dataset # huggingface datasets

In [4]:
dataset_wikipedia_en = load_dataset("wikipedia", "20220301.en")

Found cached dataset wikipedia (/Users/seven/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset = dataset_wikipedia_en
split_dataset = dataset["train"].train_test_split(test_size=0.0005, seed=2357, shuffle=True)

Loading cached split indices for dataset at /Users/seven/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559/cache-6cea40a8991b1426.arrow and /Users/seven/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559/cache-4b3c5592cb6498e7.arrow


In [6]:
print(split_dataset)
print(split_dataset['train'])

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6455440
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 3230
    })
})
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6455440
})


In [7]:
print(len(split_dataset['train']))

6455440


In [8]:
import threading

class ThreadSafeCounter:
    def __init__(self):
        self.count = 0
        self.lock = threading.Lock()

    def increment(self, value: int):
        with self.lock:
            self.count += value

    def get_count(self):
        with self.lock:
            return self.count

In [9]:
from tqdm.auto import trange, tqdm

total_len = len(dataset_wikipedia_en['train'])
print(total_len)
total_progress = tqdm(total=total_len)

train = split_dataset['train']
enc = tiktoken.get_encoding("gpt2")

counter = ThreadSafeCounter()

def process_batch(examples):
    ids = enc.encode_batch([example for example in examples['text']])
    eos_tokens = [enc.eot_token] * len(ids)
    ids = [id + [eos_token] for id, eos_token in zip(ids, eos_tokens)]
    lens = [len(id) for id in ids]
    out = {'ids': ids, 'len': lens}
    counter.increment(batch_size)
    total_progress.update(total_len/counter.get_count())
    total_progress.refresh()
    return out

# tokenize the dataset using batch processing
tokenized = dataset_wikipedia_en.map(
    process_batch,
    batched=True,
    batch_size=batch_size,
    remove_columns=['text'],
    num_proc=num_proc,
    desc="tokenizing the dataset",
)


6458670


  0%|          | 0/6458670 [00:00<?, ?it/s]

Parameter 'function'=<function process_batch at 0x144ed9240> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


tokenizing the dataset (num_proc=8):   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [10]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'ids', 'len'],
        num_rows: 6458670
    })
})

In [11]:
use_slow = False
if use_slow:
    enc = tiktoken.get_encoding("gpt2")
    def process(example):
        ids = enc.encode_ordinary(example['text']) # encode_ordinary ignores any special tokens
        ids.append(enc.eot_token) # add the end of text token, e.g. 50256 for gpt2 bpe
        # note: I think eot should be prepended not appended... hmm. it's called "eot" though...
        out = {'ids': ids, 'len': len(ids)}

        batch_size = 1
        counter.increment(batch_size)
        total_progress.update(total_len/counter.get_count())
        total_progress.refresh()

        return out

    tokenized = split_dataset.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=num_proc,
    )

In [12]:
import os
try:
    folder = os.path.dirname(__file__)
except:
    folder = os.getcwd()
folder

'/Users/seven/Documents/ai/nanoGPT'

In [13]:
tokenized.items()

dict_items([('train', Dataset({
    features: ['id', 'url', 'title', 'ids', 'len'],
    num_rows: 6458670
}))])

In [17]:
import numpy as np
import os
from tqdm import tqdm

# # concatenate all the ids in the dataset into one large file we can use for training
# concatenate all the ids in each dataset into one large file we can use for training
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'])
    filename = os.path.join(os.path.dirname(__file__), f'{split}.bin')
    dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))

    print(f"writing {filename}...")
    idx = 0
    for example in tqdm(dset):
        arr[idx : idx + example['len']] = example['ids']
        idx += example['len']
    arr.flush()


writing /Users/seven/Documents/ai/nanoGPT/train.bin...


ValueError: could not broadcast input array from shape (8817,) into shape (1712,)

In [ ]:
print('done ')